In [57]:
cd code

/home/ai/Desktop/Pre-IGR-F/code


In [68]:
import numpy as np
import torch.nn as nn
import torch
from torch.autograd import grad
from math import pi
import trimesh
from scipy.spatial import cKDTree
from model.sample import Sampler


In [84]:
## Network.py
def get_learning_rate_schedules(self, schedule_specs):

    schedules = []

    for schedule_specs in schedule_specs:

        if schedule_specs["Type"] == "Step":
            schedules.append(
                utils.StepLearningRateSchedule(
                    schedule_specs["Initial"],
                    schedule_specs["Interval"],
                    schedule_specs["Factor"],
                )
            )

        else:
            raise Exception(
                'no known learning rate schedule of type "{}"'.format(
                    schedule_specs["Type"]
                )
            )

    return schedules

def gradient(inputs, outputs):
    d_points = torch.ones_like(outputs, requires_grad=False, device=outputs.device)
    points_grad = grad(
        outputs=outputs,
        inputs=inputs,
        grad_outputs=d_points,
        create_graph=True,
        retain_graph=True,
        only_inputs=True, allow_unused = True)[0][:, -3:]
    return points_grad


class ImplicitNet(nn.Module):
    def __init__(
            self,
            f_T = False,
            k=3,
            d_in=3,
            dims=[512,512,512,512,512,512,512,512],
            skip_in={4},
            geometric_init=True,
            radius_init=1,
            beta=100
            
    ):
        super().__init__()
       
        self.f_T = f_T
        
        if f_T:
            self.B_gause  = torch.randn(size=(256,3))*k
        
        if f_T:
            dims = [512] + dims + [1]
        else:
            dims = [d_in] + dims + [1]
    
        self.num_layers = len(dims)
        self.skip_in = skip_in

        for layer in range(0, self.num_layers - 1):

            if layer + 1 in skip_in:
                out_dim = dims[layer + 1] - d_in
            else:
                out_dim = dims[layer + 1]

            lin = nn.Linear(dims[layer], out_dim)

            # if true preform preform geometric initialization
            if geometric_init:

                if layer == self.num_layers - 2:

                    torch.nn.init.normal_(lin.weight, mean=np.sqrt(np.pi) / np.sqrt(dims[layer]), std=0.00001)
                    torch.nn.init.constant_(lin.bias, -radius_init)
                else:
                    torch.nn.init.constant_(lin.bias, 0.0)

                    torch.nn.init.normal_(lin.weight, 0.0, np.sqrt(2) / np.sqrt(out_dim))

            setattr(self, "lin" + str(layer), lin)

        if beta > 0:
            self.activation = nn.Softplus(beta=beta)

        # vanilla relu
        else:
            self.activation = nn.ReLU()

    def forward(self, input):

        x = input
        
        if self.f_T:
            x_proj = (2.*np.pi*data)@self.B_gause.T
            x = torch.cat([torch.sin(x_proj), torch.cos(x_proj)], axis=-1)


        for layer in range(0, self.num_layers - 1):

            lin = getattr(self, "lin" + str(layer))

            if layer in self.skip_in:
                 x = torch.cat([x, input], -1) / np.sqrt(2)

            x = lin(x)

            if layer < self.num_layers - 2:
                x = self.activation(x)

        return x

In [87]:
# optimizer = torch.optim.Adam(
#     [
#         {
#             "params": mlp_model.parameters(),
#             "lr": self.lr_schedules[0].get_learning_rate(0),
#             "weight_decay": 0
#         },
#     ])

In [71]:
mlp_model = ImplicitNet()

In [26]:
def load_point_cloud_by_file_extension(file_name  = "/home/ai/Desktop/Preimage_Implicit_DLTaskData/armadillo_10000.xyz"):

    ext = file_name.split('.')[-1]

    if ext == "npz" or ext == "npy":
        point_set = torch.tensor(np.load(file_name)).float()
    else:
        point_set = torch.tensor(trimesh.load(file_name, "xyz").vertices).float()

    return point_set

In [77]:
data = load_point_cloud_by_file_extension().requires_grad_()

In [37]:
sigma_set = []
ptree = cKDTree(data)

for p in np.array_split(data, 100, axis=0):
    d = ptree.query(p, 50 + 1)
    sigma_set.append(d[0][:, -1])

sigmas = np.concatenate(sigma_set)
local_sigma = torch.from_numpy(sigmas).float()
local_sigma.shape

torch.Size([10098])

In [78]:
d_in=3
indices = torch.tensor(np.random.choice(data.shape[0], 2048, False))
cur_data = data[indices]
mnfld_pnts = cur_data[:, : d_in]
mnfld_sigma = local_sigma[indices]

In [81]:
global_sigma = 1.8
grad_lambda = 1
sampler = Sampler.get_sampler("NormalPerPoint")(global_sigma, local_sigma)

In [86]:
nonmnfld_pnts = sampler.get_points(mnfld_pnts.unsqueeze(0), mnfld_sigma.unsqueeze(0)).squeeze()

# forward pass

mnfld_pred = mlp_model(mnfld_pnts)
nonmnfld_pred = mlp_model(nonmnfld_pnts)

# compute grad

mnfld_grad = gradient(mnfld_pnts, mnfld_pred)
nonmnfld_grad = gradient(nonmnfld_pnts, nonmnfld_pred)

# manifold loss

mnfld_loss = (mnfld_pred.abs()).mean()

# eikonal loss

grad_loss = ((nonmnfld_grad.norm(2, dim=-1) - 1) ** 2).mean()

loss = mnfld_loss + grad_lambda * grad_loss

# normals loss

if False:
    normals = cur_data[:, -self.d_in:]
    normals_loss = ((mnfld_grad - normals).abs()).norm(2, dim=1).mean()
    loss = loss + self.normals_lambda * normals_loss
else:
    normals_loss = torch.zeros(1)

loss
# # back propagation

# optimizer.zero_grad()

# loss.backward()

# optimizer.step()

tensor(0.3724, grad_fn=<AddBackward0>)